In [6]:
import numpy as np
import time
import os
import random
import scipy.io as sio
from sklearn import neighbors #, datasets, manifold
from sklearn.model_selection import KFold
from scipy import linalg
from scipy.linalg import eigh
import matplotlib
import matplotlib.pyplot as plt

In [7]:
# # Neighbor search
# we will implement K-nearest neighbor search
def knn_Mat(X, K, t=2.0, dist_metric="euclidean", algorithm="ball_tree"):
    """ compute the neighborhood matrix

    Keyword arguments:
        X: np.array of data, each row(not column) is associated with one car. (A car is represented by a vector)
        K: number of neighbors to seek for each element (the element itself is excluded)
        dist_metric: with which metric to compute the distances
        algorithm: which which algorithm to use

    return:
        the neighborhood matrix
    """

    n, p = X.shape  # n: number of elements, p: dimension (number of data per element)

    knn = neighbors.NearestNeighbors(K + 1, metric=dist_metric, algorithm=algorithm).fit(X)
    distances, nbors = knn.kneighbors(X)  # return k nearest neighbours of each member of X, nbors is array of indices

    return (nbors[:, 1:])  # neighborhood matrix first one is the point itself, which should be excluded


# # calculation of reconstruction weights
def get_weights(X, nbors, reg, K):
    """ compute the weight matrix

    Keyword arguments:
        X: np.array of data, each row is associated with one car
        nbors; neighborhood matrix
        regularized term: avoid the covariance matrix to be singular #regularizer
        K: number of neighbours selected for each car

    return:
        the weight matrix
    """

    n, p = X.shape  # n number of car shapes, n rows and p columns (60 rows and 112000 columns in our case)

    Weights = np.zeros((n, n))

    for i in range(n):

        X_bors = X[nbors[i], :] - X[i]  # Gi in MLLE paper
        cov_nbors = np.dot(X_bors, X_bors.T)

        # regularization terms
        trace = np.trace(cov_nbors)
        if trace > 0:
            R = reg * trace
        else:
            R = reg
        cov_nbors.flat[::K + 1] += R  # [::K+1] -> every K+1 step. Allows adding only on trace
        weights = linalg.solve(cov_nbors, np.ones(K).T, sym_pos=True)  # solution to cov_nbors*weights = 1
        # optimal solution to min(cov_nbors*W)

        # normalizing
        weights = weights / weights.sum()
        # only ponderable weights of neighbours should be different from zero
        #Weights[i, nbors[i]] = weights # put all those weights into a matrix
        Weights[i, nbors[i]] = weights

    return (Weights)


#### Calculate the embedded data using the weights
# calculation of the new embedding
def Y_(Weights, d):
    """ reconstruct data into the low dimensional space

    Keyword arguments:
        Weights: weight matrix
        d: number of dimension in which to project onto (2 in our case)

    return:
        the dataset projected onto d dimensions
    """
    n, p = Weights.shape
    I = np.eye(n)
    m = (I - Weights)
    M = m.T.dot(m)

    eigvals, eigvecs = eigh(M, eigvals=(1, d), overwrite_a=True)
    ind = np.argsort(np.abs(eigvals))

    return (eigvecs[:, ind])  # we only keep the the most important vectors (?? xl)


def LLE_(X, K, d, nbors):
    """ proceed to various LLE steps

    Keyword arguments:
        X: dataset
        K: number of neighbours to seek
        d: on how many dimension to project the dataset

    return:
        the data set projected into d dimensions
    """
    reg = 0.001 # regularized term

    try:
        if nbors == None:
            nbors = knn_Mat(X, K)
    except:
        pass
    Weights = get_weights(X, nbors, reg, K)

    Y = Y_(Weights, d)

    return [Y, Weights, nbors]

# knn_new: neighbors of the new shape in the manifold space
## Newly added data functions
def knn_new_(X, K, x_new, t=2.0, dist_metric="euclidean", algorithm="ball_tree"):
            """ compute the neighborhood matrix

            Keyword arguments:
                X: np.array of data, each row(not column) is associated with one car. (A car is represented by a vector)
                K: number of neighbors to seek for each element (the element itself is excluded)
                dist_metric: with which metric to compute the distances
                algorithm: which which algorithm to use

            return:
                the neighborhood matrix
            """

            n, p = X.shape  # n: number of elements, p: dimension (number of data per element)

            knn = neighbors.NearestNeighbors(K + 1, metric=dist_metric, algorithm=algorithm).fit(X)
            nbors = knn.kneighbors([x_new],K + 1, return_distance=False)  # return k nearest neighbours of each member of X, nbors is array of indices

            return (nbors[:, 1:])  # neighborhood matrix first one is the point itself, which should be excluded

#### calculation of reconstruction weights
def get_weights_new(x_new, knn_new, K, reg=0.001):
    """ compute the weight matrix

    Keyword arguments:
        x_new: a new vector representing a new car shape
        knn_new: neighborhood of the new car shape
        reg: avoid the covariance matrix to be singular #regularizer
        K: number of neighbours selected for each car

    return:
        the vector of weights for neighboring data points
    """

    X_bors = X[knn_new[0:K], :] - x_new  # Gi in MLLE paper, X[i] = X[i,:] ith row of X
    cov_nbors = np.dot(X_bors, X_bors.T)

    # regularization terms
    trace = np.trace(cov_nbors)
    if trace > 0:
        R = reg * trace
    else:
        R = reg
    cov_nbors.flat[::K + 1] += R  # [::K+1] -> every K+1 step. Allows adding only on trace
    weights = linalg.solve(cov_nbors, np.ones(K).T, sym_pos=True)  # solution to cov_nbors*weights = 1
        # optimal solution to min(cov_nbors*W)

        # normalizing
    weights = weights / weights.sum()

    return (weights)

In [8]:
#read vectorization data and Cd data
fmat = sio.loadmat('ex4_160_latentVecs_cds.mat')
# print(fmat.keys())

vec_dim = 128
X_SPVAE_Vecs = fmat['vecs'][:140]
y_cds_GT = fmat['cds'][0][:140]

X_test = fmat['vecs'][140:]
y_test = fmat['cds'][0][140:]

In [9]:
#k-fold cross validation
for fold in [5, 10, 20, 140]:
    kfold = KFold(fold, True, 123)

    itr = 0
    f1 = open(f'Experiments_dim' + f'{vec_dim}' + f'_LLE_{fold}fold_val.txt', 'a')
    f2 = open(f'Experiments_dim' + f'{vec_dim}' + f'_LLE_{fold}fold_test.txt', 'a')

    for train, val in kfold.split(X_SPVAE_Vecs):
        X = X_SPVAE_Vecs[train]
        n_sample, sample_dim = np.shape(X)
        K = 6 #K=6 gets the lowest reconstruction error
        d = 2 #dimensionality
        nbors = None
        Y, Weights, nbors = LLE_(X, K, d, nbors)
        # err = np.linalg.norm(Y - np.dot(Weights,Y),'fro')**2 # reconstruction error for Y (low dimensional data)

        # extracting the weights of neighbors for all shapes
        W_shapes = np.zeros((n_sample,K)) # W_shapes: weights extracted from Weight matrix
        for i in range(n_sample):
            for j in range(K):
                n = nbors[i,:][j]
                W_shapes[i,j] = Weights[i,n]

        X_val = X_SPVAE_Vecs[val]
        for i, x_val in enumerate(X_val):
            # Neighbors search for a new added shape
            # we will implement K-nearest neighbor search
            knn_val = knn_new_(X,K,x_val)[0] #knn_new(X,K,x_new)is a tuple with one element an array
            w_new = get_weights_new(x_val, knn_val, K)
            # Calculation of the drag coefficient of the validation cars
            cds_nn = y_cds_GT[knn_val] #NN's cds
            cd_new_pred = np.dot(w_new , cds_nn)
            cd_GT = y_cds_GT[val[i]]
            error_cd_new_pred = cd_new_pred - cd_GT #prediction error
            f1.writelines(f'Dim_{vec_dim}, val:{val[i]}, GT:{cd_GT}, pred: {cd_new_pred}, error:{error_cd_new_pred} \n')

        #20 test cars
        for j, x_test in enumerate(X_test):
            # Neighbors search for a new added shape
            # we will implement K-nearest neighbor search
            knn_test = knn_new_(X,K,x_test)[0] #knn_new(X,K,x_new)is a tuple with one element an array
            w_new = get_weights_new(x_test, knn_test, K)
            # Calculation of the drag coefficient of the validation cars
            cds_nn = y_cds_GT[knn_test] #NN's cds
            cd_new_pred = np.dot(w_new , cds_nn)
            cd_GT = y_test[j]
            error_cd_new_pred = cd_new_pred - cd_GT
            f2.writelines(f'Dim_{vec_dim}, test:{j}, GT:{cd_GT}, pred: {cd_new_pred}, error:{error_cd_new_pred} \n')

        # error_cd_new_pred_all = np.array(error_cd_new_pred_all)
        print(f"{fold}-fold, itr {itr} is done!!")
        itr += 1

    f1.close()
    f2.close()


C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True, random_state=123 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword

5-fold, itr 0 is done!!
5-fold, itr 1 is done!!
5-fold, itr 2 is done!!


C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From versi

5-fold, itr 3 is done!!
5-fold, itr 4 is done!!
10-fold, itr 0 is done!!


C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From versi

...

C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From versi

140-fold, itr 132 is done!!
140-fold, itr 133 is done!!
140-fold, itr 134 is done!!
140-fold, itr 135 is done!!


C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From versi

140-fold, itr 136 is done!!
140-fold, itr 137 is done!!
140-fold, itr 138 is done!!
140-fold, itr 139 is done!!


C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
C:\Users\Xingang Li\.conda\envs\sdf_lle\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=7 as keyword args. From versi